In [36]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import warnings

import numpy as np

from tqdm import tnrange, tqdm_notebook
import multiprocessing as mp

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import NullFormatter, FormatStrFormatter
from matplotlib import rcParams

from astropy.table import Table, Column, join

from riker.galaxy import GalaxyMap
from riker.data import BeneMassAgeZMaps

from kungpao.display import display_single

# Color maps
IMG_CMAP = plt.get_cmap('Greys')
IMG_CMAP.set_bad(color='w')

warnings.filterwarnings("ignore")

plt.rc('text', usetex=True)
rcParams.update({'axes.linewidth': 1.5})
rcParams.update({'xtick.direction': 'in'})
rcParams.update({'ytick.direction': 'in'})
rcParams.update({'xtick.minor.visible': 'True'})
rcParams.update({'ytick.minor.visible': 'True'})
rcParams.update({'xtick.major.pad': '7.0'})
rcParams.update({'xtick.major.size': '8.0'})
rcParams.update({'xtick.major.width': '1.5'})
rcParams.update({'xtick.minor.pad': '7.0'})
rcParams.update({'xtick.minor.size': '4.0'})
rcParams.update({'xtick.minor.width': '1.5'})
rcParams.update({'ytick.major.pad': '7.0'})
rcParams.update({'ytick.major.size': '8.0'})
rcParams.update({'ytick.major.width': '1.5'})
rcParams.update({'ytick.minor.pad': '7.0'})
rcParams.update({'ytick.minor.size': '4.0'})
rcParams.update({'ytick.minor.width': '1.5'})
rcParams.update({'axes.titlepad': '10.0'})
rcParams.update({'font.size': 25})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
def map_list(hdf5, sample, data_type='mass_gal', proj='xy', 
             index_col='index', index_list=False):
    """Gather a lit of maps for a subsample of galaxies."""
    if index_list:
        indices = sample
    else:
        indices = list(sample[index_col])
    
    return [hdf5.get_maps(idx, proj, maps_only=True)[data_type] for idx in indices]


def sample_mosaic(sample, maps, n_col=5, img_size=3, zmin=3.5, zmax=10.5,
                  show_mstar=True, show_mhalo=True, info=None, output=None):
    """Making a mosaic figure of a subsample."""
    n_row = int(np.ceil(len(sample) / n_col))

    fig = plt.figure(figsize=(n_col * img_size, n_row * img_size))
    fig.subplots_adjust(
        left=0.0, right=1.0, bottom=0.0, top=1.0, wspace=0.00, hspace=0.00)

    gs = GridSpec(n_row, n_col)
    gs.update(wspace=0.0, hspace=0.00)

    for ii, data in enumerate(maps):
        galaxy = sample[ii]

        # Show the aperture
        ax = plt.subplot(gs[ii])
        ax.yaxis.set_major_formatter(NullFormatter())
        ax.xaxis.set_major_formatter(NullFormatter())
        if ii == 0:
            ax = display_single(
                data, ax=ax, stretch='log10', zmin=zmin, zmax=zmax,
                cmap=IMG_CMAP, no_negative=True, color_bar=True, scale_bar=False,
                color_bar_color='k')
        else:
            ax = display_single(
                data, ax=ax, stretch='log10', zmin=zmin, zmax=zmax,
                cmap=IMG_CMAP, no_negative=True, color_bar=False, scale_bar=False,
                color_bar_color='k')
 
        # Using color to separate central v.s. satellites
        if galaxy['cen_flag']:
            text_color = 'r'
        else:
            text_color = 'k'
        
        if info is not None:
            if show_mstar:
                ax.text(0.70, 0.13, r'$%5.2f$' % galaxy[info], fontsize=18, 
                        transform=ax.transAxes, c=text_color)
            else:
                ax.text(0.70, 0.06, r'$%5.2f$' % galaxy[info], fontsize=18, 
                        transform=ax.transAxes, c=text_color)

        if show_mstar:
            ax.text(0.70, 0.06, r'$%5.2f$' % galaxy['logms'], fontsize=18, 
                    transform=ax.transAxes, c=text_color)
            
        if show_mhalo:
            ax.text(0.07, 0.06, r'$%5.2f$' % galaxy['logm200c'], fontsize=18, 
                    transform=ax.transAxes, c=text_color)
    
    if output is not None:
        fig.savefig(output, dpi=110)
        plt.close(fig)
    else:
        return fig

### Show mosaic picture for a sample of galaxies

In [2]:
# Input HDF5 from simulation
tng_dir = '/Users/song/data/massive/simulation/riker/tng'

# HDF5 files
tng_file = os.path.join(tng_dir, 'galaxies_tng100_072_agez_highres.hdf5')

tng_label = 'tng100_z0.4_hres'

tng_data = BeneMassAgeZMaps(tng_file, label=tng_label)

# Get the summary table for all galaxies in the data
tng_galaxies = tng_data.sum_table()

* Here we use the 3-D shape measurements of these TNG halos as examples
    - `T_50` is the triaxiality parameter at 50 kpc.
    - `haloId` is the `catsh_id`.
* We will match the catalog to our galaxy catalog, and group them into very prolate (`T_50 > 0.8`) and very oblate (`T_50 < 0.3`) galaxies, and show their stellar mass ranked mosaics.

In [6]:
tng_t50 = Table.read('/Users/song/data/massive/simulation/riker/tng/tng100_T50.csv')

tng_t50.rename_column('haloId', 'catsh_id')

tng_3dshape = join(tng_galaxies, tng_t50, keys='catsh_id', join_type='inner')

#### Both central and satellites, sorted by stellar mass

In [58]:
tng_pro = tng_3dshape[tng_3dshape['T_50'] >= 0.79]
tng_obl = tng_3dshape[tng_3dshape['T_50'] <= 0.35]

tng_pro.sort('logms')
tng_obl.sort('logms')

print(len(tng_pro), len(tng_obl))

mgal_pro = map_list(tng_data, tng_pro)
mgal_obl = map_list(tng_data, tng_obl)

77 76


In [59]:
_ = sample_mosaic(tng_pro, mgal_pro, info='T_50', 
                  output='tng100_hres_t50_gt_0.79_mass.png')

_ = sample_mosaic(tng_obl, mgal_obl, info='T_50', 
                  output='tng100_hres_t50_lt_0.35_mass.png')

#### Both central and satellites, sorted by `T50`

In [63]:
tng_pro_t = tng_3dshape[tng_3dshape['T_50'] >= 0.79]
tng_obl_t = tng_3dshape[tng_3dshape['T_50'] <= 0.35]

tng_pro_t.sort('T_50')
tng_obl_t.sort('T_50')

print(len(tng_pro_t), len(tng_obl_t))

mgal_pro_t = map_list(tng_data, tng_pro_t)
mgal_obl_t = map_list(tng_data, tng_obl_t)

77 76


In [64]:
_ = sample_mosaic(tng_pro_t, mgal_pro_t, info='T_50', 
                  output='tng100_hres_t50_gt_0.79_t50.png')

_ = sample_mosaic(tng_obl_t, mgal_obl_t, info='T_50', 
                  output='tng100_hres_t50_lt_0.35_t50.png')

#### Central only, sorted by stellar mass

In [76]:
tng_pro_cen = tng_3dshape[(tng_3dshape['T_50'] >= 0.80) & 
                          np.asarray(tng_3dshape['cen_flag'])]
tng_obl_cen = tng_3dshape[(tng_3dshape['T_50'] <= 0.38) & 
                          np.asarray(tng_3dshape['cen_flag'])]

tng_pro_cen.sort('logms')
tng_obl_cen.sort('logms')

print(len(tng_pro_cen), len(tng_obl_cen))

mgal_pro_cen = map_list(tng_data, tng_pro_cen)
mgal_obl_cen = map_list(tng_data, tng_obl_cen)

62 63


In [77]:
_ = sample_mosaic(tng_pro_cen, mgal_pro_cen, info='T_50', 
                  output='tng100_hres_t50_gt_0.79_cen_mass.png')

_ = sample_mosaic(tng_obl_cen, mgal_obl_cen, info='T_50', 
                  output='tng100_hres_t50_lt_0.35_cen_mass.png')

#### Central only, sorted by `T_50`

In [78]:
tng_pro_cen_t = tng_3dshape[(tng_3dshape['T_50'] >= 0.80) & 
                            np.asarray(tng_3dshape['cen_flag'])]
tng_obl_cen_t = tng_3dshape[(tng_3dshape['T_50'] <= 0.38) & 
                            np.asarray(tng_3dshape['cen_flag'])]

tng_pro_cen_t.sort('T_50')
tng_obl_cen_t.sort('T_50')

print(len(tng_pro_cen_t), len(tng_obl_cen_t))

mgal_pro_cen_t = map_list(tng_data, tng_pro_cen_t)
mgal_obl_cen_t = map_list(tng_data, tng_obl_cen_t)

62 63


In [79]:
_ = sample_mosaic(tng_pro_cen_t, mgal_pro_cen_t, info='T_50', 
                  output='tng100_hres_t50_gt_0.79_cen_t50.png')

_ = sample_mosaic(tng_obl_cen_t, mgal_obl_cen_t, info='T_50', 
                  output='tng100_hres_t50_lt_0.35_cen_t50.png')